# Stress Test性能测试
- 千帆 Python SDK 提供了基于locust工具的对大模型服务进行快速压测以及性能评估的功能。该功能入口在Dataset对象的stress_test方法中。

- 本文提供了一个使用multi_stress_test进行压测的示例，通过本文，您可以快速上手我们的多轮压测工具。

- 由于Locust中使用了gevent库来保证高并发性能，而gevent的高并发依赖于monkey patching的非阻塞I/O机制，但该机制在Notebook环境中默认未开启。因此，在开始测试前，需要进行monkey patching操作。这样做是为了确保整个环境，包括IPython/Jupyter自己的组件，都使用gevent兼容的非阻塞版本，从而避免因混合使用阻塞和非阻塞操作导致的不一致性和潜在的死锁问题。

In [1]:
from gevent import monkey
monkey.patch_all()

True

开始压测：

In [4]:
import os

# 开启压测服务
os.environ['QIANFAN_ENABLE_STRESS_TEST'] = "true"

from qianfan.dataset import Dataset

# 请在此处填写您的ak、sk
os.environ["QIANFAN_ACCESS_KEY"] = "..."
os.environ["QIANFAN_SECRET_KEY"] = "..."

# 需要初始化一个数据集对象
ds = Dataset.load(data_file="./data_file/stress_test.jsonl")

ds.multi_stress_test(
    origin_users=2,# 初始并发
    # workers=2,# 实际工作的进程数
    # spawn_rate=2,# 生成user的速度
    # model="ERNIE-Speed-8K",
    # model_type="ChatCompletion", 
    endpoint="YourEndpoint",# 压测服务的endpoint
    rounds = 5,# 压测轮次
    interval=2,# 轮次间加压数
    first_latency_threshold=5,# 单位秒
    # round_latency_threshold=20,# 单位秒
    # success_rate_threshold=99.5,# 成功率阈值，百分制
)

[INFO][2024-07-18 11:37:39.841] dataset.py:430 [t:4370825664]: no data source was provided, construct
[INFO][2024-07-18 11:37:39.863] dataset.py:276 [t:4370825664]: construct a file data source from path: ./data_file/stress_test.jsonl, with args: {}
[INFO][2024-07-18 11:37:39.875] file.py:291 [t:4370825664]: use format type FormatType.Jsonl
[INFO][2024-07-18 11:37:39.890] utils.py:359 [t:4370825664]: start to get memory_map from /Users/jianruitian/.qianfan_cache/dataset/Users/jianruitian/Documents/GitHub/bce-qianfan-sdk/cookbook/dataset/data_file/stress_test.arrow
[INFO][2024-07-18 11:37:39.901] utils.py:284 [t:4370825664]: has got a memory-mapped table
Type     Name                                                                          # reqs      # fails |    Avg     Min     Max    Med |   req/s  failures/s
--------|----------------------------------------------------------------------------|-------|-------------|-------|-------|-------|-------|--------|-----------
--------|-------

与stress_test不同的是，multi_stress_test的输出有更多的轮次，每轮发压都会展示一次压测结果。

multi_stress_test的前端报告也与stresstest的前端报告不同，我们添加了对各个压测轮次的压测数据的展示以及对压测模型信息的展示。前端报告的样例见cookbook/dataset/data_file/performance_table.html。

以下是各参数的说明：

- **workers (int)**：指定发压使用的worker数目，每个worker为1个进程，默认为1个进程；
- **origin_users (int)**：指定发压使用的初始user数，必须大于worker数目；每个worker负责模拟${users}/${workers}个虚拟用户；
- **runtime (str)**：指定发压任务的最大运行时间，格式为带时间单位的字符串，例如（300s, 20m, 3h, 1h30m）；压测任务启动后会一直运行到数据集内所有数据都请求完毕，或到达该参数指定的最大运行时间；该参数默认值为'0s'，表示不设最大运行时间；
- **spawn_rate (int)**：指定每秒启动的user数目；
- **model (str)**：指定需要压测服务的模型名称。该参数与endpoint只能指定一个；
- **endpoint (str)**：指定需要压测服务的url路径。该参数与model只能指定一个；
- **model_type (str)**：指定被测服务的模型类型。 目前只支持'ChatCompletion'与'Completion两类'；默认值为'ChatCompletion'；
- **hyperparameters (Optional[Dict[str, Any]])**：指定压测时使用的超参数；
- **rounds (int)**：指定压测轮数；
- **interval (int)**：指定压测轮之间的加压并发数，比如interval=2，则在第1轮压测结束后，会在第2轮开始时，额外启动两个user的并发，以此类推；
- **first_latency_threshold (float)**：指定首句时延的阈值，超过该阈值会停止在本轮压测，单位为秒；
- **round_latency_threshold (float)**：指定全长时延的阈值，超过该阈值会停止在本轮压测，单位为秒；
- **success_rate_threshold (float)**：指定请求成功率的阈值，低于该阈值会停止在本轮压测，单位为百分比；

各项指标含义如下：

- **user_num**: 压测使用的本轮user数，即本轮发压的并发数；
- **worker_num**: 压测使用的worker数目，即进程数；
- **spawn_rate**: 每秒启动的user数目；
- **model_type**: 被压测服务的模型类型；
- **hyperparameters**: 压测使用的超参数；
- **QPS**：服务每秒实际处理的请求数；
- **Latency Avg/Min/Max/50%/80%**：全长时延的平均值/最小值/最大值/50分位值/80分位值；
- **FirstTokenLatency Avg/Min/Max/50%/80%**：首句时延的平均值/最小值/最大值/50分位值/80分位值；
- **InputTokens Avg**：单次请求输入的token长度平均值；
- **OutputTokens Avg**：单次请求输出的token长度平均值；
- **TotalQuery/SuccessQuery/FailureQuery**：总请求数/成功请求数/失败请求数；
- **TotalTime**：总运行时间；
- **SuccessRate**：请求成功率；